In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

import config as c
from pipeline.Layer import Layer
from utils.cuda import turn_off_gpu
from models.svm.svm import SVM
from models.keras_dense_classifier.keras_dense_classifier import KerasDenseClassifier as KDC
from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix

turn_off_gpu()
init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('train.csv', sep=',')
pd.set_option('display.max_columns', 500)
df['addr_region_fact_encoding2'] = (df['addr_region_fact_encoding2']*11).round(0).astype(int)
df['addr_region_fact_encoding1'] = (df['addr_region_fact_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding1'] = (df['addr_region_reg_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding2'] = (df['addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding2'] = (df['app_addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding1'] = (df['app_addr_region_reg_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding1'] = (df['app_addr_region_fact_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding2'] = (df['app_addr_region_fact_encoding2']*11).round(0).astype(int)
df['app_addr_region_sale_encoding1'] = (df['app_addr_region_sale_encoding1']*39).round(0).astype(int)
df['app_addr_region_sale_encoding2'] = (df['app_addr_region_sale_encoding2']*7).round(0).astype(int)

df=df[list(df.columns[:10]) + ['fo']]
df = df.drop(['card_id', 'addr_region_reg', 'addr_region_fact'], axis=1)

categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
numerical_columns   = [c for c in df.columns if df[c].dtype.name != 'object']
df.head()

,target,delivery_type,channel_name,channel_name_2,channel_name_modified_2018,sas_limit_after_003_amt,sas_limit_last_amt,fo
0,1,cat_1,cat_0,cat_3,cat_0,1,1,0
1,0,cat_1,cat_2,cat_5,cat_2,3,3,0
2,1,cat_1,cat_0,cat_3,cat_0,1,1,4
3,0,cat_1,cat_0,cat_3,cat_0,3,3,0
4,0,cat_1,cat_0,cat_3,cat_0,0,0,0


In [3]:
df = pd.concat([df[numerical_columns], pd.get_dummies(df[categorical_columns])], axis=1)

In [4]:
X = df[df.columns[1:]].to_numpy()
y = df['target'].to_numpy()

In [5]:
kds = KDC()
print(kds.fit_ensemble(10, 5, X, y, KDC.default_model_constructor_parameters))

71/71 [==============================] - 0s 367us/step - loss: 0.2339 - accuracy: 0.6081
0.613302218914032


In [ ]:
kds.predict(X)